In [1]:
# !pip install SpeechRecognition
import speech_recognition as sr

r = sr.Recognizer()
sr.Microphone.list_microphone_names()
mic = sr.Microphone(device_index=1)

from win32com.client import constants, Dispatch

speaker = Dispatch("SAPI.SpVoice")

from stockfish import Stockfish

stockfish = Stockfish("C://Users//anupk//Downloads//stockfish_14_win_x64//stockfish_14_x64")

In [2]:
from random import random
import re 
import sys
import pickle

from IPython.display import clear_output

regex = '([a-h])([1-8])([a-h])([1-8])'

def check(move):
    global speaker
    
    if move == 'stop':
        return -1
    
    if move == 'save':
        game_name = input("enter the name to save the game")
        
        game_name = game_name + '.p'
        game_moves.append(player_color)
        game_moves.append(player_turn)
        pickle.dump( game_moves, open( game_name, "wb" ) )
        sys.exit()
    
    if stockfish.is_move_correct(move):
        
        if(re.search(regex,move)):

            speaker.Speak(move)
            return 1
        
        else:
            speaker.Speak("speak clearly!")
            return 0
    else:
        speaker.Speak("invalid move")
        return 0
    

def config_engine(elo = 1000, skill_lvl = 12, depth = 15):
    stockfish.set_skill_level(skill_lvl)
    stockfish.set_elo_rating(elo)
    stockfish.set_depth(depth)


def choose_player_color():
    if random() > 0.5:
        return 'white'
    else:
        return 'black'
    
def human_move(speak_flag):
    
    if speak_flag == True:
        print("call out")
        global speaker
        speaker.Speak("Your turn")
        try:
            with mic as source:
                audio = r.listen(source)
            h_move = r.recognize_google(audio)
            h_move = h_move.replace(" ","")
            h_move = h_move.lower()
            valid = check(h_move)

            if valid == 1:
                return h_move

            elif valid == -1:
                sys.exit()
                
            else:
                human_move()
        except:
            speaker.Speak("speak clearly!!!")
            human_move()
    
    else:
        print("command please")
        h_move = input("Enter your move: ")
        valid = check(h_move)
        if valid == 1:
            return h_move
        
        elif valid == -1:
            sys.exit()
        
        else:
            speaker.Speak("check the format")
            human_move(False)
            

def engine_move():
    e_move = stockfish.get_best_move()
    speaker.Speak(e_move)
    return e_move



def display_current_positions(moves):
    stockfish.set_position(moves)
    clear_output(wait=True)
    print(moves)
    viz = stockfish.get_board_visual().split('\n')
    viz.append('--A---B---C---D---E---F---G---H--')
    for i in viz:
        print(i)
    


In [3]:
player_turn = -1
player_color = "color"
game_moves = []
voice = False
import time


def play_game():
    global voice,game_moves,player_color,player_turn
    rating = input("set elo rating: [default : 1000]")
    if rating:
        config_engine(elo =rating)
        
    old_or_new = input("do you wanna play old game or new game")
    
    if old_or_new == "old":
        game_name = input("enter the name of the game saved")
        try:
            loaded_game = pickle.load(open( game_name+".p", "rb" ))
        except Exception as e:
            print(e)
            
        player_turn = loaded_game.pop()
        player_color = loaded_game.pop()
        game_moves = loaded_game 
        
    else:
        player_color = choose_player_color()
        player_turn = 1 if player_color == "white" else 0

    print("you are playing as ",player_color)
    speaker.Speak("you are playing as "+player_color)
    play_type = input("enter voice or key : ")
    voice = True if play_type == "voice" else False
    
    while True:
        try:
            display_current_positions(game_moves)
            if player_turn == 1 :
                hm_mv = human_move(voice)
                if hm_mv != None:
                    game_moves.append(hm_mv)
                    display_current_positions(game_moves)
                    player_turn = 0
                
            if player_turn == 0:
                game_moves.append(engine_move())
                display_current_positions(game_moves)
                player_turn = 1
                
        except Exception as e:
            print(e)  

In [ ]:
play_game()